In [48]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [49]:
pasta = '/mnt/HD_500GB_WD/00_Doutorado/Codigos_Notebooks_Medias_Resample/Dataframes_Media1m/'

### Abrindo os .pkl com medias 1min:
### Propriedades ópticas:
Bs_550_1 = pd.read_pickle(pasta+'1ogrenM1_Bs_G_Dry_1um_Neph3W_1_df_T_dias_raw_1m.pkl')
aod = pd.read_pickle(pasta+'AOD_AERONET_processado.pkl')

In [50]:
datasets = [Bs_550_1, aod]
nomes = ['Bs_550_1', 'AOD']
labels = [u'$\\beta$$_e$$_s$$_p$$_.$', 'AOD']

In [51]:
### Função para padronizar o formato do índice (minutos) e das datas nas colunas (padrão: 'DD/MM/AAAA'):
def padroniza_datas(df):
    mins = np.arange(0, 1440, 1)
    datas = list(df.columns)
    datas_arrumadas = []
    for dia in datas:
        if len(dia)==10:
            datas_arrumadas.append(dia)
        elif len(dia)==8:
            ltemp = []
            for c in dia:
                ltemp.append(c)
            ltemp.insert(0,'0')
            ltemp.insert(3,'0')
            datas_arrumadas.append(''.join(ltemp))
        elif len(dia)==9 and dia[1]=='/':
            ltemp = []
            for c in dia:
                ltemp.append(c)
            ltemp.insert(0,'0')
            datas_arrumadas.append(''.join(ltemp))
        elif len(dia)==9 and dia[1]!='/':
            ltemp = []
            for c in dia:
                ltemp.append(c)
            ltemp.insert(3,'0')
            datas_arrumadas.append(''.join(ltemp))
    df.columns = datas_arrumadas
    df.index = mins
    df.index.name = "tempo"
    df.columns.name = "dias"
    
### Chamando a função padroniza_datas:
for ds, nome in zip(datasets, nomes):
    padroniza_datas(ds)

In [52]:
### Função que constrói o dataframe com as datas do GoAmazon (01/01/14 a 31/12/15) cheio de NaN's. Depois disso
### insere os dias de 2014 e de 2015 faltantes nos dataframes que contém as estatísticas dos dados:
def completa_dias_goamazon(df):
    days = pd.date_range(start="2014-01-01", end='2015-12-31',freq='D').strftime('%d/%m/%Y')
    mins = np.arange(0, 1440, 1)
    df_nans = pd.DataFrame(columns=days, index=mins)    
    booleanos = []
    for dia in list(df_nans.columns):
        booleanos.append(dia in list(df.columns))
    for i in range(len(booleanos)):
        if booleanos[i] == True:     # Há dados neste dia
            pass
        elif booleanos[i] == False:  # Não há dados neste dia
            df.insert(i, column=df_nans.columns[i], value=df_nans.T.iloc[i].values)

In [53]:
### Chamando a função:
for ds, nome in zip(datasets, nomes):
    completa_dias_goamazon(ds)
    
print(len(datasets[0].T))
print(len(datasets[1].T))

730
730


In [54]:
### Média Referências - Beta:
# Limpo:
limp = ['11/02/2014','17/02/2014','23/02/2014','24/02/2014',
        '26/02/2014','28/02/2014','02/03/2014','06/03/2014',
        '07/03/2014','08/03/2014','12/03/2014','20/03/2014',
        '21/03/2014','22/03/2014','25/03/2014','28/03/2014',
        '25/02/2015', '03/05/2015', '04/05/2015']
# listona = []
# selec = datasets[0][limp]
# for col in selec.columns:
#     for v in selec[col]:
#         listona.append(v)
# mean_ref_limpo_beta = np.nanmean(listona)
# print(mean_ref_limpo_beta)

# # Poluído:
# listona = []
# selec = datasets[0][['11/09/2014', '28/09/2014']]
# for col in selec.columns:
#     for v in selec[col]:
#         listona.append(v)
# mean_ref_pol_beta = np.nanmean(listona)
# print(mean_ref_pol_beta)

#########################################################
### Média Referências - AOD AERONET:
# Limpo:
listona = []
selec = datasets[1][limp]
for col in selec.columns:
    for v in selec[col]:
        listona.append(v)
mean_ref_limpo_aod = np.nanmean(listona)
print(mean_ref_limpo_aod)

# Poluído:
listona = []
selec = datasets[1][['11/09/2014', '28/09/2014']]
for col in selec.columns:
    for v in selec[col]:
        listona.append(v)
mean_ref_pol_aod = np.nanmean(listona)
print(mean_ref_pol_aod)

0.06876418421052631
0.2017999444444444


In [55]:
# ### Média Períodos 2014 - Beta:
# # Limpo:
# listona = []
# selec = datasets[0].T[10:90]
# for col in selec.columns:
#     for v in selec[col]:
#         listona.append(v)
# mean_limpo14_beta = np.nanmean(listona)
# print(mean_limpo14_beta)
# # Poluído:
# listona1 = []
# listona2 = []
# selec1 = datasets[0].T[210:277]
# selec2 = datasets[0].T[300:345]
# for col in selec1.columns:
#     for v in selec1[col]:
#         listona1.append(v)
# for col in selec2.columns:
#     for v in selec2[col]:
#         listona2.append(v)
# listona = listona1+listona2
# mean_pol14_beta = np.nanmean(listona)
# print(mean_pol14_beta)

### Média Períodos 2014 - AOD AERONET:
# Limpo:
listona = []
selec = datasets[1].T[10:90]
for col in selec.columns:
    for v in selec[col]:
        listona.append(v)
mean_limpo14_aod = np.nanmean(listona)
print(mean_limpo14_aod)
# Poluído:
listona1 = []
listona2 = []
selec1 = datasets[1].T[210:277]
selec2 = datasets[1].T[300:345]
for col in selec1.columns:
    for v in selec1[col]:
        listona1.append(v)
for col in selec2.columns:
    for v in selec2[col]:
        listona2.append(v)
listona = listona1+listona2
mean_pol14_aod = np.nanmean(listona)
print(mean_pol14_aod)

0.09026209311740893
0.27754531418918926


In [56]:
# ### Variação percentual limpo Beta 2014:
# print(((mean_limpo14_beta - mean_ref_limpo_beta)/mean_ref_limpo_beta)*100)

# ### Variação percentual poluido Beta 2014:
# print(((mean_pol14_beta - mean_ref_pol_beta)/mean_ref_pol_beta)*100)

In [57]:
### Variação percentual limpo AOD 2014:
print(((mean_limpo14_aod - mean_ref_limpo_aod)/mean_ref_limpo_aod)*100)

### Variação percentual poluido AOD 2014:
print(((mean_pol14_aod - mean_ref_pol_aod)/mean_ref_pol_aod)*100)

31.263235583607425
37.5348813664305


In [58]:
### Sensibilidade AOD limpo 14 = Média da IRF_diária/VPA_limpo_AOD:
print(3.79/31.263235583607425)

### Sensibilidade AOD pol 14 = Média da IRF_diária/VPA_pol_AOD:
print(1.53/37.5348813664305)

0.12122865497604636
0.04076208434132319


In [59]:
##################################################

In [60]:
### Média Períodos 2015 - AOD AERONET:
corte15 = datasets[1].T[365:].T
# Limpo:
listona = []
selec = corte15.T[0:210]
for col in selec.columns:
    for v in selec[col]:
        listona.append(v)
mean_limpo15_aod = np.nanmean(listona)
print(mean_limpo15_aod)
# Poluído:
listona = []
selec = corte15.T[265:335]
for col in selec.columns:
    for v in selec[col]:
        listona.append(v)
mean_pol15_aod = np.nanmean(listona)
print(mean_pol15_aod)

0.1245338613138686
0.5873089475524476


In [61]:
### Variação percentual limpo AOD 2015:
print(((mean_limpo15_aod - mean_ref_limpo_aod)/mean_ref_limpo_aod)*100)

### Variação percentual poluido AOD 2015:
print(((mean_pol15_aod - mean_ref_pol_aod)/mean_ref_pol_aod)*100)

81.10279754443033
191.0352374820073


In [62]:
### Sensibilidade AOD limpo 15 = Média da IRF_diária/VPA_limpo_AOD:
print(1.09/81.10279754443033)

### Sensibilidade AOD pol 15 = Média da IRF_diária/VPA_pol_AOD:
print(0.79/191.0352374820073)

0.013439733683697757
0.004135362723719525
